# Antibody levels buildings with fixed households

Check whether antibody levels in a building are more similar than one would expect statistically, if people still stay in the same household.

Here, we implement an approximate permutation by only permuting households of the same size.

## Data preprocessing

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from copy import deepcopy
import multiprocessing as mp
from jabbar import jabbar
from pyprojroot import here
import os

import sys
base_path = str(here("", project_files=[".here"]))
perm_path = os.path.join(base_path, "PermutationStudies")
if perm_path not in sys.path:
    sys.path.insert(0, perm_path)
from src.functions import *

%matplotlib inline

random.seed(0)
np.random.seed(0)

# old or new cutoff
cutoff = '%%cutoff%%'
# control variable of interest
var = 'address_id'
# measurements to study
data_key = '%%data_key%%'
# number of permutations
n_perm = %%n_perm%%

# identifier
id_ = f"bd_{cutoff}_{data_key}_{n_perm}"

In [ ]:
blab = read_blab(base_path)

geo = read_geo(base_path)

# restrict to study households
geo = geo[geo.hht_ID.isin(blab.hh_id.unique())]

# merge
geo = geo.rename(columns={'hht_ID': 'hh_id'})
data = pd.merge(blab, geo)
print(blab.shape, geo.shape, data.shape, "initially")

# remove duplicate columns
data = data.drop_duplicates(subset=['ind_id'], keep='first')
print(data.shape, "after remove duplicates")

# remove nans   
data = data[data[data_key + '_quant'].notnull()]
print(data.shape, "after remove nans")

# to categorical
if cutoff == "cutold":
    cutoffs = get_old_cutoffs()
elif cutoff == "cutnew":
    cutoffs = get_new_cutoffs(base_path)
data[data_key + '_quant'] = (data[data_key + '_quant'] >= cutoffs[data_key]).astype(float)

# data plot
fig, ax = plt.subplots(figsize=(3, 3))
ax.hist(data[data_key + '_quant'], color='C0', bins=100)
ax.set_xlabel(data_key)
ax.set_ylabel("Frequency")
fig.tight_layout()

Convert to numpy for efficiency

In [ ]:
%%time
data_arr = get_data_arr(data, data_key)
var_ids, var_ids_uq, var_id_matrix, var_sizes = get_var_id_stuff(data, var)
data_matrix = create_data_matrix(data_arr, var_id_matrix)
# control
print(data_matrix.shape)

In [ ]:
plt.hist(var_sizes, bins=np.arange(var_sizes.max()+1)+1, align='left')
plt.title("Building size distribution")
plt.xlabel("Building size [participants]")
plt.ylabel("Number")

## Household aware permutation function

In [ ]:
%%time
hh_ids = np.array(data.hh_id)
hh_ids_uq = np.array(data.hh_id.unique())

# list of array of indices belonging to the same household
ixs_by_hh = [np.where(hh_ids == hh_id)[0] for hh_id in hh_ids_uq]

# sort by size
ixs_by_hh_size = {}
for hh_ixs in ixs_by_hh:
    ixs_by_hh_size.setdefault(len(hh_ixs), []).append(hh_ixs)

In [ ]:
# checks
perm_ixs = permute_ixs_fix_hhs(len(hh_ids), ixs_by_hh_size)
for hh_ixs in ixs_by_hh:
    all_hh_ids = [hh_ids[perm_ixs[hh_ix]] for hh_ix in hh_ixs]
    assert len(set(all_hh_ids)) == 1

## Define statistics

Since we perform a time-intensive permutation test, we need to make the computations efficient. To check validity and efficiency, below we provide implementations in pandas, numpy, and fully vectorized numpy.

Pandas implementation:

In [ ]:
%%time
real_mean = statistic_mean_pd(data, data_key, var, var_ids_uq)
print(real_mean)
real_variance = statistic_var_pd(data, data_key, var, var_ids_uq)
print(real_variance)

Numpy implementation with iteration:

In [ ]:
%%time
real_mean = statistic_mean_np_iter(data_arr, var_ids, var_ids_uq)
print(real_mean)
real_variance = statistic_var_np_iter(data_arr, var_ids, var_ids_uq)
print(real_variance)

Completely vectorized numpy implementation:

In [ ]:
%%time
data_matrix_dict = create_data_matrix(data_arr, var_id_matrix)

real_mean = statistic_mean(data_matrix, var_sizes)
print(real_mean)
real_variance = statistic_var(data_matrix, var_sizes, var_id_matrix)
print(real_variance)

This >30 fold speed-up should be enough for the moment.

## Permutation test

In [ ]:
%%time

# set random seed for reproducibility
np.random.seed(0)

# define permutations
data_arr_perms = get_data_arr_perms_fix_hhs(
    data_arr, n_perm, ixs_by_hh_size)

# for results
means = []
variances = []

# loop over all permutations
for data_arr_perm in jabbar(data_arr_perms, symbols='🦄'):
    # get permutation
    data_matrix = create_data_matrix(data_arr_perm, var_id_matrix)
    # compute mean mean and mean variance
    means.append(statistic_mean(data_matrix, var_sizes))
    variances.append(statistic_var(data_matrix, var_sizes, var_id_matrix))

# to numpy arrays
means = np.array(means)
variances = np.array(variances)

In [ ]:
#save data
save_data(id_, means=means, variances=variances,
          real_mean=real_mean, real_variance=real_variance,
          perm_path=perm_path)

## Analysis

In [ ]:
# load data
means, variances, real_mean, real_variance = load_data(
    id_=id_, obj_keys=['means', 'variances', 'real_mean', 'real_variance'],
    perm_path=perm_path)

In [ ]:
# plot for means
plot_kde(samples=means, obj_key='means', real_sample=real_mean,
         data_key=data_key, id_=id_, suptitle="Average mean over buildings",
         perm_path=perm_path)
plot_hist(samples=means, obj_key='means', real_sample=real_mean,
          data_key=data_key, id_=id_, suptitle="Average mean over buildings",
          perm_path=perm_path)

In [ ]:
# plot for variances
plot_kde(samples=variances, obj_key='variances', real_sample=real_variance,
         data_key=data_key, id_=id_, suptitle="Average variance over buildings",
         perm_path=perm_path)
plot_hist(samples=variances, obj_key='variances', real_sample=real_variance,
          data_key=data_key, id_=id_, suptitle="Average variance over buildings",
          perm_path=perm_path)

In [ ]:
print("Percentiles:")
print("Mean", data_key, sum(means <= real_mean) / len(means))
print("Variance", data_key, sum(variances <= real_variance) / len(variances))